In [1]:
#Importing libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import re
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#Ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
hotel_json = {}
hotel_json["reviews"]=[]
review = []

def Scrap_reviews(url_list):
    for url in url_list:
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        html = soup.prettify("utf-8")
        hotel_json = {}
        hotel_json["reviews"]=[]
        review = []

        for line in soup.find_all('script',attrs={"type" : "application/ld+json"}):
            details = line.text.strip()
            details = details.replace('/','')
            details = json.loads(details)
            hotel_json["name"] = details["name"]
            hotel_json["url"] = details["url"]
            hotel_json["image"] = details["image"]
            details["priceRange"] = details["priceRange"]
            hotel_json["priceRange"] = details["priceRange"]
            hotel_json["aggregateRating"]={}
            hotel_json["aggregateRating"]["ratingValue"] = details["aggregateRating"]["ratingValue"]
            hotel_json["aggregateRating"]["reviewCount"] = details["aggregateRating"]["reviewCount"]
            hotel_json["address"]={}
            hotel_json["address"]["Street"]=details["address"]["streetAddress"]
            hotel_json["address"]["Locality"]=details["address"]["addressLocality"]
            hotel_json["address"]["Region"]=details["address"]["addressRegion"]
            hotel_json["address"]["Zip"]=details["address"]["postalCode"]
            
        for line in soup.find_all('div', attrs = {'class', "c-review-snippet"}):
            review = line.text.strip()
            review = review.replace('\r', ' ').replace('\n', ' ')
            review = ' '.join(review.split())
            hotel_json["reviews"].append(review)
            
        # Nota2: aixo no! que es la info en format json  com a tal ;)
        with open(hotel_json["name"]+".json", 'w') as outfile:
            json.dump(hotel_json, outfile, indent=4)    

In [3]:
#Here we copy the urls that we want to scrap from Booking.com
url_list = [
'https://www.booking.com/hotel/at/courtyard-by-marriott-wien-messe.html?aid=356980;label=gog235jc-1DCAsoRkIIYmFsbW9yYWxIM1gDaEaIAQGYATG4AQfIAQzYAQPoAQH4AQKIAgGoAgO4AtLI7vMFwAIB;sid=2a31209c8df7e74f89ef392e4d85314e;dest_id=-1995499;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1585161769;srpvid=3dd0839412ab0023;type=total;ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/at/vienna-stephansdom.en-gb.html?aid=356980;label=gog235jc-1DCAsoDkISdmllbm5hLXN0ZXBoYW5zZG9tSDNYA2hGiAEBmAEJuAEHyAEM2AED6AEBiAIBqAIDuALnv630BcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dist=0&group_adults=2&group_children=0&keep_landing=1&no_rooms=1&sb_price_type=total&type=total&',
'https://www.booking.com/hotel/fr/lesaintgregoire.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-1456928&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416553&srpvid=babe7af47d9e0094&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/gb/london-city-suites-by-montcalm.en-gb.html?label=gen173nr-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSAlYBGhGiAEBmAEJuAEHyAEM2AED6AEBiAIBqAIDuALHkrH0BcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dist=0&group_adults=2&group_children=0&keep_landing=1&no_rooms=1&sb_price_type=total&type=total&',
'https://www.booking.com/hotel/fr/crayon-rouge.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-1456928&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416410&srpvid=e4707aac7e530079&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/fr/best-western-opa-c-ra-grands-boulevards.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-1456928&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416446&srpvid=e4707abe85b6006e&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/gb/st-george-39-s-wembley.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-2601889&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416479&srpvid=df597acf77bb0088&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/fr/hotel-le-mathurin.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-1456928&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416576&srpvid=242c7affe55c0059&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/fr/saintpaul.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-1456928&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416611&srpvid=459d7b111365004f&type=total&ucfs=1&#hotelTmpl',    
'https://www.booking.com/hotel/gb/the-justin-james-ltd.en-gb.html?aid=356980;label=gog235jc-1DCAsoUEIebG9uZG9uLWNpdHktc3VpdGVzLWJ5LW1vbnRjYWxtSApYA2hGiAEBmAEKuAEHyAEM2AED6AEB-AECiAIBqAIDuAKbkf7zBcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dest_id=-2601889&dest_type=city&group_adults=2&group_children=0&hapos=1&hpos=1&no_rooms=1&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1585416651&srpvid=b87a7b258caa007e&type=total&ucfs=1&#hotelTmpl',
'https://www.booking.com/hotel/gb/knightsbridge-london1.en-gb.html?label=gen173nr-1DCAsoUEIVa25pZ2h0c2JyaWRnZS1sb25kb24xSAlYBGhGiAEBmAEJuAEHyAEM2AED6AEBiAIBqAIDuAKIlrH0BcACAQ;sid=4feb7fec272a84fddd9aa8104b413c8c;dist=0&group_adults=2&group_children=0&keep_landing=1&no_rooms=1&sb_price_type=total&type=total&'
]

In [4]:
#We call the function - it generates .json files with the name, address, pricing and reviews of the hotels
Scrap_reviews(url_list)